In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils import *
import torchvision.datasets as datasets
import time

In [3]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [4]:
model = torchvision.models.vgg16_bn()

In [5]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root='./data', train=True, transform=transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, 4),
        transforms.ToTensor(),
        normalize,
    ]), download=True),
    batch_size=128, shuffle=True,
    num_workers=5, pin_memory=True)

val_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root='./data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ])),
    batch_size=128, shuffle=False,
    num_workers=5, pin_memory=True)

# define loss function (criterion) and pptimizer
criterion = nn.CrossEntropyLoss()

Files already downloaded and verified


In [6]:
optimizer = torch.optim.Adam(model.parameters(), 3e-4,
                                weight_decay=0.0005)

In [11]:
def train(train_loader, model, criterion, optimizer, epoch):
    """
        Run one train epoch
    """
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    writer = SummaryWriter(flush_secs=10)

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):

        # measure data loading time
        data_time.update(time.time() - end)

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        output = output.float()
        loss = loss.float()
        # measure accuracy and record loss
        prec1 = accuracy(output.data, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 100 == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                      epoch, i, len(train_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses, top1=top1))
        writer.add_scalar('Loss/Train', losses.val)
        writer.add_scalar('Prec@1', top1.val)

def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res
def validate(val_loader, model, criterion):
    """
    Run evaluation
    """
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (input, target) in enumerate(val_loader):
        with torch.no_grad():
            output = model(input)
            loss = criterion(output, target)

        output = output.float()
        loss = loss.float()

        # measure accuracy and record loss
        prec1 = accuracy(output.data, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 200 == 0:
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                      i, len(val_loader), batch_time=batch_time, loss=losses,
                      top1=top1))

    print(' * Prec@1 {top1.avg:.3f}'
          .format(top1=top1))

    return top1.avg

In [8]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [13]:
train_epochs = 100
for i in range(train_epochs):
    train(train_loader, model, criterion, optimizer, i)


Epoch: [0][0/391]	Time 7.874 (7.874)	Data 0.101 (0.101)	Loss 1.7114 (1.7114)	Prec@1 36.719 (36.719)
Epoch: [0][100/391]	Time 5.403 (5.439)	Data 0.001 (0.002)	Loss 1.5967 (1.7350)	Prec@1 42.188 (31.815)
Epoch: [0][200/391]	Time 5.404 (12.739)	Data 0.001 (0.001)	Loss 1.7425 (1.6785)	Prec@1 38.281 (35.094)
Epoch: [0][300/391]	Time 5.380 (10.308)	Data 0.001 (0.001)	Loss 1.4242 (1.6290)	Prec@1 41.406 (37.928)
Epoch: [1][0/391]	Time 7.571 (7.571)	Data 0.105 (0.105)	Loss 1.5292 (1.5292)	Prec@1 48.438 (48.438)
Epoch: [1][100/391]	Time 5.429 (5.438)	Data 0.001 (0.002)	Loss 1.1797 (1.3392)	Prec@1 64.062 (52.351)
Epoch: [1][200/391]	Time 5.425 (5.430)	Data 0.001 (0.001)	Loss 1.4233 (1.3097)	Prec@1 51.562 (53.642)
Epoch: [1][300/391]	Time 5.259 (5.394)	Data 0.001 (0.001)	Loss 1.1209 (1.2652)	Prec@1 59.375 (55.388)
Epoch: [2][0/391]	Time 7.103 (7.103)	Data 0.123 (0.123)	Loss 1.1033 (1.1033)	Prec@1 60.156 (60.156)
Epoch: [2][100/391]	Time 5.521 (5.439)	Data 0.001 (0.002)	Loss 0.8324 (1.0826)	Prec@1 

KeyboardInterrupt: 

In [14]:
validate(val_loader, model, criterion)

Test: [0/79]	Time 4.204 (4.204)	Loss 0.3814 (0.3814)	Prec@1 89.844 (89.844)
 * Prec@1 82.690


82.69

In [39]:
model_wo = torchvision.models.vgg11()
optimizer_wo = torch.optim.Adam(model_wo.parameters(), 3e-4,
                                weight_decay=0.0005)

In [46]:
train(train_loader, model_wo, criterion, optimizer_wo, 10000)

Epoch: [10000][0/391]	Time 5.464 (5.464)	Data 0.072 (0.072)	Loss 1.1889 (1.1889)	Prec@1 57.812 (57.812)
Epoch: [10000][50/391]	Time 3.341 (3.419)	Data 0.001 (0.002)	Loss 1.1646 (1.2292)	Prec@1 54.688 (56.020)
Epoch: [10000][100/391]	Time 3.523 (3.383)	Data 0.001 (0.001)	Loss 1.1371 (1.2004)	Prec@1 57.812 (57.356)
Epoch: [10000][150/391]	Time 3.304 (3.403)	Data 0.001 (0.001)	Loss 0.9491 (1.1710)	Prec@1 60.938 (58.371)
Epoch: [10000][200/391]	Time 3.264 (3.386)	Data 0.001 (0.001)	Loss 1.1725 (1.1639)	Prec@1 57.031 (58.660)
Epoch: [10000][250/391]	Time 3.291 (3.365)	Data 0.001 (0.001)	Loss 1.0466 (1.1535)	Prec@1 58.594 (59.008)
Epoch: [10000][300/391]	Time 3.266 (3.359)	Data 0.001 (0.001)	Loss 1.1317 (1.1394)	Prec@1 60.156 (59.406)
Epoch: [10000][350/391]	Time 3.211 (3.353)	Data 0.001 (0.001)	Loss 1.1335 (1.1278)	Prec@1 56.250 (59.894)


In [45]:
print('validation_error: ' , get_model_error(model, val_loader))

test error: 0.446


TypeError: not all arguments converted during string formatting